# Lab 5: Leveraging Open Data from Wikipedia for LLM Prompt Engineering

## Overview
This lab demonstrates how to extract structured data from Wikipedia pages and use it to create effective prompts for Large Language Models (LLMs). You'll learn to work with real-world financial data, process it programmatically, and engineer prompts for various AI tasks.

## Learning Objectives
- ✓ Extract financial index components from Wikipedia
- ✓ Retrieve company infobox data programmatically
- ✓ Build structured datasets from semi-structured web data
- ✓ Design effective LLM prompts for different tasks
- ✓ Process and clean text data for AI consumption
- ✓ Create reusable prompt templates and utilities

## Part 1: Data Extraction from Wikipedia

### What is a Financial Index?
A financial index is a composite measure of a subset of companies in a specific market or sector. Examples include:
- **S&P 500**: 500 largest US companies
- **EURO STOXX 50**: 50 largest Eurozone companies
- **DAX**: 40 largest German companies

### Your Task
1. **Identify components**: Extract the list of companies in each index from Wikipedia
2. **Gather company data**: Retrieve detailed information (infoboxes) from each company's Wikipedia page
3. **Build a dataset**: Combine all data into structured format suitable for LLM processing
4. **Engineer prompts**: Create effective prompts that leverage this data for AI tasks

### Data Sources
- **Index components**: Wikipedia articles listing index members
- **Company data**: Wikipedia infoboxes (structured data boxes on company pages)
- **Dump file**: Optional - for advanced analysis of full Wikipedia articles

### Optional: Full Wikipedia Dump
For advanced analysis, you can download the complete Wikipedia dump from:
- **Link**: https://dumps.wikimedia.org/enwiki/
- **File**: `enwiki-latest-pages-articles-multistream-index.txt.bz2`
- **Use case**: Full-text search, article history analysis, or complete data scraping
- **Note**: Very large files (100+ GB) - requires significant storage and processing power

For this lab, we'll focus on extracting specific data via the Wikipedia API, which is more efficient.

In [1]:
!uv pip install pandas pathlib typing tqdm wptools loguru numpy

Using Python 3.12.12 environment at: /usr
Resolved 15 packages in 272ms
Prepared 6 packages in 130ms
Installed 6 packages in 17ms
 + html2text==2025.4.15
 + loguru==0.7.3
 + pathlib==1.0.1
 + pycurl==7.45.7
 + typing==3.10.0.0
 + wptools==0.4.17


In [2]:
# ============================================================================
# IMPORTS & SETUP
# ============================================================================
# These libraries enable us to work with Wikipedia data

import pandas as pd              # Data manipulation and analysis
import urllib.request           # HTTP requests to Wikipedia
from pathlib import Path        # Cross-platform file path handling
from typing import Union, Dict  # Type hints for better code clarity
from tqdm import tqdm          # Progress bars for long operations
import wptools               # Wikipedia parsing (infobox extraction)
from loguru import logger       # Enhanced logging
import json                     # Working with JSON data
import numpy as np             # Numerical operations
import re                       # Regular expressions for text cleaning

## Step 1: Extract Index Components from Wikipedia

### Task: Extract Company Lists
We'll extract the list of companies that make up each financial index directly from Wikipedia.

### Indices We're Covering:
1. **S&P 500** (USA) - 500 largest US companies
2. **EURO STOXX 50** (Europe) - 50 largest Eurozone companies  
3. **CAC 40** (France) - 40 largest French companies
4. **DAX** (Germany) - 40 largest German companies
5. **CSI 300** (China) - 300 largest Chinese companies
6. **S&P Latin America 40** (Latin America) - 40 major LA companies
7. **BSE SENSEX** (India) - 30 largest Indian companies
8. **NASDAQ-100** (USA Tech) - 100 largest non-financial NASDAQ companies

### How It Works:
- Each index has a Wikipedia article with a table listing its components
- We'll use `pd.read_html()` to extract all tables from these pages
- Tables are saved as CSV files for later processing
- This approach is fast, requires no authentication, and respects Wikipedia's terms

In [3]:
# ============================================================================
# FUNCTION 1: Extract Tables from Wikipedia
# ============================================================================
# This function downloads tables from Wikipedia articles and saves them locally

def get_index_components(wiki_url: str, save_dir: Union[str, Path],
                         opener: urllib.request.OpenerDirector) -> None:
    """
    Extract all HTML tables from a Wikipedia page and save as CSV files.

    Parameters:
    -----------
    wiki_url : str
        The Wikipedia page URL to scrape (e.g., list of index components)
    save_dir : Union[str, Path]
        Directory where CSV files will be saved
    opener : urllib.request.OpenerDirector
        Custom URL opener with proper User-Agent headers

    Output:
    -------
    Creates CSV files named table_0.csv, table_1.csv, etc. in save_dir
    Each file contains one table from the Wikipedia page

    Example:
    --------
    >>> get_index_components(
    ...     "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies",
    ...     "./data/indices/sp500",
    ...     opener
    ... )
    """
    save_path = Path(save_dir)
    save_path.mkdir(parents=True, exist_ok=True)

    # Install the opener so that pandas.read_html uses it under the hood
    urllib.request.install_opener(opener)

    # Let pandas handle downloading + HTML parsing from the URL
    tables = pd.read_html(wiki_url)

    # Export each table to CSV
    for idx, df in enumerate(tables):
        out_file = save_path / f"table_{idx}.csv"
        df.to_csv(out_file, index=False)

    logger.info(
        "Saved %d tables from %s into directory %s",
        len(tables),
        wiki_url,
        save_path
    )

In [4]:
# ============================================================================
# SETUP: Configure Wikipedia Index URLs and HTTP Headers
# ============================================================================

# Dictionary mapping index names to their Wikipedia article URLs
# These URLs contain tables with the company components of each index
indices = {
    "sp500": "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies",
    "eurostoxx50": "https://en.wikipedia.org/wiki/EURO_STOXX_50",
    "cac40": "https://en.wikipedia.org/wiki/CAC_40",
    "dax": "https://en.wikipedia.org/wiki/DAX",
    "csi300": "https://en.wikipedia.org/wiki/CSI_300_Index",
    "spla40": "https://en.wikipedia.org/wiki/S%26P_Latin_America_40",
    "bsesensex": "https://en.wikipedia.org/wiki/BSE_SENSEX",
    "nasdaq100": "https://en.wikipedia.org/wiki/Nasdaq-100",
}

# IMPORTANT: Configure HTTP headers to identify our bot to Wikipedia
# This is REQUIRED for ethical web scraping - identify yourself!
# Wikipedia may block requests without proper User-Agent headers

opener = urllib.request.build_opener()
opener.addheaders = [
    ("User-Agent", "MyResearchBot/1.0 (contact@example.com)")  # Identify your bot
]
urllib.request.install_opener(opener)

In [5]:
# ============================================================================
# EXECUTION: Download Index Components
# ============================================================================
# Loop through each index and extract its company components from Wikipedia
# This may take a few minutes depending on internet speed

for index_name, wiki_url in tqdm(indices.items(), desc="Downloading indices"):
    save_dir = Path(f"./data/indices/{index_name}")
    get_index_components(wiki_url, save_dir, opener)

2025-11-29 17:51:28.568 | INFO     | __main__:get_index_components:47 - Saved %d tables from %s into directory %s


## Step 2: Extract Company Infoboxes from Wikipedia

### What are Infoboxes?
Wikipedia infoboxes are structured data boxes that appear on the right side of articles. They contain:
- Company name and alternative names
- Industry classification
- Founded date and location
- Key executives
- Headquarters location
- Number of employees
- Revenue and financial metrics
- Official website URLs
- Stock exchange listings
- And much more...

### Why Infoboxes?
- **Structured data**: Unlike article body text, infoboxes are semi-structured
- **Consistency**: Fields follow a template across similar articles
- **Ease of extraction**: Wikipedia APIs can parse infoboxes directly
- **Rich context**: Perfect for LLM prompts - contains exactly the info LLMs need

### Process
1. Use the `wptools` library to fetch each company's Wikipedia page
2. Extract the infobox (structured data) from the page parse
3. Save as JSON for flexibility and later processing
4. Handle errors gracefully (some companies may not have Wikipedia pages)

In [6]:
# ============================================================================
# EXAMPLE: Extract a Single Company Infobox
# ============================================================================
# This example shows the process for one company (3M) from S&P 500
# In production, we'd loop this for all companies


def save_company_infobox(company_name: str,
                         save_dir: Union[str, Path] = "./data/infoboxes/sp500") -> None:
    """
    Fetch a company's Wikipedia infobox and save it as JSON.

    Parameters
    ----------
    company_name : str
        Name of the company as used on Wikipedia (e.g., "3M").
    save_dir : Union[str, Path], optional
        Directory where the infobox JSON will be stored.
    """
    save_path = Path(save_dir)
    save_path.mkdir(parents=True, exist_ok=True)

    try:
        # Build and parse the Wikipedia page in one chained call
        page = wptools.page(company_name, silent=True).get_parse()

        # Safely retrieve the infobox dict (fallback to empty if missing)
        infobox = page.data.get("infobox") or {}

        # Construct JSON output path
        json_file = save_path / f"{company_name}.json"
        with json_file.open("w", encoding="utf-8") as f:
            json.dump(infobox, f, indent=2, ensure_ascii=False)

        logger.info("Infobox for %s saved to %s", company_name, json_file)

    except Exception as exc:
        logger.error("Error extracting infobox for %s: %s", company_name, exc)


# Example usage:
save_company_infobox("3M", "./data/infoboxes/sp500")

2025-11-29 17:51:34.395 | INFO     | __main__:save_company_infobox:35 - Infobox for %s saved to %s


In [7]:
# ============================================================================
# DISPLAY: View the Extracted Infobox
# ============================================================================
# This shows what data we extracted from Wikipedia

# Load and display the saved infobox

def show_infobox(company_name: str, base_dir: str = "./data/infoboxes/sp500") -> None:
    """
    Load and display a company's infobox from a JSON file.
    """
    json_path = Path(base_dir) / f"{company_name}.json"

    try:
        with json_path.open("r", encoding="utf-8") as f:
            infobox_data = json.load(f)
    except FileNotFoundError:
        print(f"Infobox file not found for {company_name}. Run the extraction step first.")
        return

    print(f"Infobox for {company_name}\n")
    print(f"Total fields extracted: {len(infobox_data)}\n")

    # Iterate over fields and print them
    for key, value in infobox_data.items():
        print(f"{key}: {value}")


# Example usage:
show_infobox("3M")

Infobox for 3M

Total fields extracted: 24

name: 3M Company
logo: 3M wordmark.svg
logo_size: 175px
image: 3-M Building Maplewood MN1.jpg
image_size: 250px
image_caption: 3M headquarters in [[Maplewood, Minnesota]]
former_name: Minnesota Mining and Manufacturing Company (1902–2002)
type: [[Public company|Public]]
traded_as: {{Unbulleted list|New York Stock Exchange|MMM|[[Dow Jones Industrial Average|DJIA]] component|[[S&P 100]] component|[[S&P 500]] component}} {{New York Stock Exchange|MMM}}
ISIN: {{ISIN|sl|=|n|pl|=|y|US88579Y1010}}
industry: [[Conglomerate (company)|Conglomerate]]
foundation: {{Start date and age|1902|6|13}} in [[Two Harbors, Minnesota]], U.S.
founders: {{Unbulleted list|J. Danley Budd|Henry S. Bryan|William A. McGonagle|John Dwan|Hermon W. Cable | Charles Simmons|ref|{{cite web |url=https://www.3m.com.au/3M/en_AU/company-au/news-releases/full-story/?storyid=51f5cfac-3ea9-4a98-a406-e2b955c3fd40 |title=It all started with a rock |date=June 11, 2021 |work=3M Australia 

## Step 3: Aggregate Infoboxes into Databases

### What We're Building
We're converting individual JSON files (one per company) into consolidated CSV databases (one per index).

### Why?
- **Easier analysis**: CSV format works with pandas, Excel, and most analysis tools
- **Efficiency**: One file per index instead of hundreds of individual JSON files
- **Standardization**: Creates a uniform dataset structure for LLM processing

### Process
1. Read all JSON infobox files for an index from disk
2. Convert each JSON to a DataFrame row
3. Concatenate all rows into a single DataFrame
4. Save as CSV with proper encoding

### Notes for Future Enhancement
- The infoboxes contain many fields beyond what we use now (URLs, images, etc.)
- Future work could extract and leverage additional information
- This foundation allows flexible data extraction later

In [8]:
# ============================================================================
# EXECUTION: Merge All Infoboxes into Index Databases
# ============================================================================
# Loop through each index folder and consolidate all JSON infoboxes into CSV

def build_infobox_databases(
    infoboxes_base: Union[str, Path] = "./data/infoboxes",
    output_dir: Union[str, Path] = "./data/databases"
) -> None:
    """
    Merge all company infobox JSON files into per-index CSV databases.

    Parameters
    ----------
    infoboxes_base : Union[str, Path]
        Base directory containing one subfolder per index
        (e.g. ./data/infoboxes/sp500, ./data/infoboxes/nasdaq100, ...).
    output_dir : Union[str, Path]
        Directory where aggregated CSV databases will be written.
    """
    infoboxes_base = Path(infoboxes_base)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    # Iterate over all subdirectories (each representing an index)
    for index_dir in infoboxes_base.iterdir():
        if not index_dir.is_dir():
            continue

        index_name = index_dir.name

        # Collect all JSON files in the index directory
        json_files: List[Path] = sorted(index_dir.glob("*.json"))
        if not json_files:
            logger.warning("No JSON files found in %s", index_dir)
            continue

        records: List[Dict] = []

        for json_file in json_files:
            try:
                # Read and parse JSON
                data = json.loads(json_file.read_text(encoding="utf-8"))
                # Store filename stem as a reference to the original company
                data["_source_file"] = json_file.stem
                records.append(data)
            except Exception as exc:
                logger.error("Failed to read %s: %s", json_file, exc)

        if not records:
            logger.warning("No valid records found in %s", index_dir)
            continue

        # Convert list of dicts to DataFrame
        df = pd.DataFrame.from_records(records)

        # Save one CSV per index
        csv_path = output_dir / f"{index_name}_infoboxes.csv"
        df.to_csv(csv_path, index=False, encoding="utf-8")

        logger.info(
            "Aggregated %d companies from %s -> %s",
            len(records),
            index_name,
            csv_path,
        )


# Example usage:
build_infobox_databases("./data/infoboxes", "./data/databases")

2025-11-29 17:53:15.966 | INFO     | __main__:build_infobox_databases:61 - Aggregated %d companies from %s -> %s


# New step : complete process for every index

In [9]:
# ============================================================================
# FUNCTION: Fetch All Company Infoboxes for Any Index
# ============================================================================
import time  # For rate limiting
# Configuration: which table and column contains company names for each index
INDEX_CONFIG: Dict[str, Dict[str, str]] = {
    "sp500":       {"table": "table_1.csv", "column": "Security"},
    "eurostoxx50": {"table": "table_4.csv", "column": "Name"},
    "cac40":       {"table": "table_4.csv", "column": "Company"},
    "dax":         {"table": "table_4.csv", "column": "Company"},
    "csi300":      {"table": "table_3.csv", "column": "Company"},
    "spla40":      {"table": "table_1.csv", "column": "Company name"},
    "bsesensex":   {"table": "table_2.csv", "column": "Company"},
    "nasdaq100":   {"table": "table_4.csv", "column": "Company"},
}


def fetch_index_infoboxes(
    index_name: str,
    indices_base: Union[str, Path] = "./data/indices",
    output_dir: Union[str, Path] = "./data/databases",
    rate_limit: float = 0.5,
) -> pd.DataFrame:
    """
    Fetch Wikipedia infoboxes for all companies in a given index.

    Parameters
    ----------
    index_name : str
        One of: "sp500", "eurostoxx50", "cac40", "dax", "csi300",
                "spla40", "bsesensex", "nasdaq100".
    indices_base : Union[str, Path], optional
        Base directory where index component CSVs are stored.
        e.g. ./data/indices/sp500/table_1.csv
    output_dir : Union[str, Path], optional
        Directory where the aggregated infobox CSV will be written.
    rate_limit : float, optional
        Sleep time (in seconds) between Wikipedia requests.

    Returns
    -------
    pd.DataFrame
        DataFrame containing all successfully fetched infoboxes.
        Also saved as ./data/databases/{index_name}_infoboxes.csv
    """
    if index_name not in INDEX_CONFIG:
        valid = ", ".join(sorted(INDEX_CONFIG.keys()))
        raise ValueError(f"Unknown index: {index_name}. Choose from: {valid}")

    config = INDEX_CONFIG[index_name]

    indices_base = Path(indices_base)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    # Load company list for this index
    table_path = indices_base / index_name / config["table"]
    if not table_path.exists():
        raise FileNotFoundError(f"Index table not found: {table_path}")

    df_companies = pd.read_csv(table_path)
    companies: List[str] = (
        df_companies[config["column"]]
        .dropna()
        .astype(str)
        .unique()
        .tolist()
    )

    logger.info("Index %s: found %d companies", index_name, len(companies))

    records: List[Dict[str, Any]] = []
    failed: List[str] = []

    for company_name in tqdm(companies, desc=f"Fetching {index_name} infoboxes"):
        try:
            # Chain creation and parsing
            page = wptools.page(company_name, silent=True).get_parse()
            infobox = page.data.get("infobox") or {}

            if infobox:
                # Keep track of original company name
                infobox["_company_name"] = company_name
                records.append(infobox)
            else:
                failed.append(company_name)
        except Exception as exc:
            failed.append(company_name)
            logger.debug("Error fetching %s: %s", company_name, exc)

        time.sleep(rate_limit)

    # Convert to DataFrame and save
    df_infoboxes = pd.DataFrame.from_records(records)
    csv_path = output_dir / f"{index_name}_infoboxes.csv"
    df_infoboxes.to_csv(csv_path, index=False, encoding="utf-8")

    logger.info(
        "Index %s: successfully fetched %d infoboxes, failed %d",
        index_name,
        len(records),
        len(failed),
    )
    if failed:
        sample_failed = failed[:10]
        suffix = "..." if len(failed) > 10 else ""
        logger.warning("Some companies failed (%d total): %s%s", len(failed), sample_failed, suffix)
    logger.info("Saved infobox database to %s", csv_path)

    return df_infoboxes

# Example usage:
df_sp500 = fetch_index_infoboxes("sp500")

2025-11-29 17:53:29.657 | INFO     | __main__:fetch_index_infoboxes:70 - Index %s: found %d companies
Fetching sp500 infoboxes:   4%|▍         | 19/502 [00:39<15:41,  1.95s/it]API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
2025-11-29 17:54:08.935 | DEBUG    | __main__:fetch_index_infoboxes:89 - Error fetching %s: %s
Fetching sp500 infoboxes:   4%|▍         | 20/502 [00:39<12:48,  1.60s/it]API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&

In [10]:
fetch_index_infoboxes("nasdaq100")

2025-11-29 18:10:17.991 | INFO     | __main__:fetch_index_infoboxes:70 - Index %s: found %d companies
Fetching nasdaq100 infoboxes:   3%|▎         | 3/102 [00:07<03:50,  2.33s/it]API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
2025-11-29 18:10:25.258 | DEBUG    | __main__:fetch_index_infoboxes:89 - Error fetching %s: %s
Fetching nasdaq100 infoboxes:   4%|▍         | 4/102 [00:07<02:47,  1.71s/it]API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia

,name,logo,image,image_upright,image_caption,trading_name,former_name,type,traded_as,industry,...,locations,members,members_year,production,defunct,fate,successor,native_name,logo_class,genre
0,Adobe Inc.,[[File:Adobe Corporate wordmark.svg|frameless|...,Adobe World Headquarters.jpg,1.1,"[[Adobe World Headquarters]] in [[San Jose, Ca...",Adobe,Adobe Systems Incorporated (1982–2018),[[Public company|Public]],{{Unbulleted list|NASDAQ|ADBE|[[Nasdaq-100]] c...,[[Software]],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Advanced Micro Devices, Inc.",[[File:AMD Logo.svg|frameless|upright=1.1|clas...,2485 Augustine Drive headquarters in Santa Cla...,1.1,"Headquarters in [[Santa Clara, California]], i...",NaN,NaN,[[Public company|Public]],{{Unbulleted list\n | |NASDAQ|AMD|\n | [[Nas...,{{ubl|[[Semiconductor industry|Semiconductor]]...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Airbnb, Inc.",Airbnb Logo Bélo.svg,"888 Brannan, San Francisco, 2016.jpg",NaN,Headquarters at 888 Brannan Street,NaN,NaN,[[Public company|Public]],{{ubl|NASDAQ|ABNB| (Class A)|[[Nasdaq-100]] co...,[[Lodging]],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"American Electric Power Company, Inc.",AEP-Logo-Red-Gray.svg,AEP Building 1.jpg,NaN,"[[AEP Building]], the company's headquarters i...",NaN,NaN,[[Public company|Public]],{{ubl|NASDAQ|AEP|[[DJUA]] component|[[Nasdaq-1...,[[Electric Utility|Electric utilities]],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Amgen Inc.,Amgen.svg,Amgenheadquarters.jpg,NaN,"Headquarters in Thousand Oaks, California",NaN,NaN,[[Public company|Public]],{{unbulleted list|NASDAQ|AMGN|[[Nasdaq-100]] c...,[[Biotechnology]],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,Vertex Pharmaceuticals Incorporated,Vertex logo.svg,NaN,NaN,NaN,NaN,NaN,[[Public company|Public]],{{ubl|NASDAQ|VRTX|[[Nasdaq-100]] component|[[S...,{{ubl|[[Pharmaceuticals]] | [[Biotherapy|Bioth...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,"Warner Bros. Discovery, Inc.",Warner Bros. Discovery.svg,NaN,NaN,WBD's headquarters in 230 [[Park Avenue South]...,NaN,NaN,[[Public company|Public]],{{ublist\n| |NASDAQ|WBD| (Series A)\n| [[Nasda...,{{ublist\n| [[Media conglomerate|Media]]\n| [[...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,NaN,Workday logo.svg,Workday Headquarters.jpg,NaN,"Headquarters in Pleasanton, California",NaN,NaN,[[Public company|Public]],{{ubl| class|=|nowrap\n | |NASDAQ|WDAY| (Class...,{{ubl|[[Cloud computing]]|[[Enterprise softwar...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,Xcel Energy Inc.,Xcel-energy.svg,ExcelEnergyDenver.jpg,NaN,"1800 Larimer, Xcel Energy Regional Headquarter...",NaN,NaN,[[Public company|Public]],{{ubl|NASDAQ|XEL|[[DJUA]] component|[[Nasdaq-1...,[[Public utility|Utilities]],...,NaN,NaN,NaN,{{ubl|Electric: 114.98 [[TWh]]|Natural Gas: 40...,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
fetch_index_infoboxes("cac40")

2025-11-29 18:13:47.117 | INFO     | __main__:fetch_index_infoboxes:70 - Index %s: found %d companies
Fetching cac40 infoboxes: 100%|██████████| 40/40 [01:29<00:00,  2.24s/it]
2025-11-29 18:15:16.916 | INFO     | __main__:fetch_index_infoboxes:98 - Index %s: successfully fetched %d infoboxes, failed %d
2025-11-29 18:15:16.916 | WARNING  | __main__:fetch_index_infoboxes:107 - Some companies failed (%d total): %s%s
2025-11-29 18:15:16.917 | INFO     | __main__:fetch_index_infoboxes:108 - Saved infobox database to %s


,name,logo,logo_alt,logo_size,foundation,founders,hq_location,hq_location_city,hq_location_country,type,...,caption,birth_date,birth_place,death_date,era,region,school_tradition,main_interests,notable_ideas,genre
0,Accor S.A.,Accor logo.svg,Accor logo,150,{{start date and age|1967|df|=|yes}} <br />[[P...,{{Unbulleted list|[[Gérard Pelisson]]|[[Paul D...,[[Tour Sequana]],[[Issy-les-Moulineaux]],France,[[Public company|Public]],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Air Liquide S.A.,"Air Liquide - logo (France, 2017).svg",NaN,250px,{{start date and age|1902}},NaN,NaN,NaN,NaN,[[S.A. (corporation)|Société Anonyme]],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Airbus SE,Airbus Logo 2017.svg {{!}} class=skin-invert,NaN,180px,NaN,NaN,"{{Indented plainlist|\n* [[Leiden]], Netherlan...",NaN,NaN,[[Public company|Public]],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ArcelorMittal S.A.,ArcelorMittal.svg,NaN,200px,{{start date and age|2007}},NaN,NaN,NaN,NaN,[[Public company|Public]],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AXA S.A.,AXA Logo.svg,NaN,165px,{{Start date and age|1921}},NaN,NaN,NaN,NaN,[[Public company|Public]],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,BNP Paribas.svg,NaN,50px,* {{start date and age|df|=|yes|1822|12|13}} C...,NaN,NaN,NaN,NaN,[[Public company|Public]],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Bouygues S.A.,Bouygues.svg,NaN,200px,{{start date and age|1952}},NaN,NaN,NaN,NaN,[[Public company|Public]],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Bureau Veritas S.A.,Bureau Veritas.svg,NaN,150px,{{start date and age|1828}},NaN,NaN,NaN,NaN,[[S.A. (corporation)|Société anonyme]],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Capgemini SE,Capgemini New logo.svg,NaN,250px,{{Start date and age|df|=|yes|1 October 1967}},NaN,NaN,NaN,NaN,[[Public company|Public]],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Carrefour S.A.,Carrefour_Groupe.svg,NaN,NaN,{{start date and age|df|=|yes|1958|1|1}},{{ubl|Marcel Fournier|[[Denis Defforey]]|Jacqu...,NaN,NaN,NaN,[[Public company]],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
fetch_index_infoboxes("spla40")

2025-11-29 18:15:16.956 | INFO     | __main__:fetch_index_infoboxes:70 - Index %s: found %d companies
Fetching spla40 infoboxes:  42%|████▎     | 17/40 [00:29<00:35,  1.54s/it]API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
2025-11-29 18:15:46.812 | DEBUG    | __main__:fetch_index_infoboxes:89 - Error fetching %s: %s
Fetching spla40 infoboxes:  62%|██████▎   | 25/40 [00:42<00:27,  1.85s/it]API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&

,name,logo,logo_size,type,traded_as,foundation,hq_location_city,hq_location_country,area_served,key_people,...,trade_name,predecessors,brands,members,net_income_year,website,founded,hq_location,production,num_locations_year
0,Ambev S.A.,Ambev logo.svg,225px,[[Public company|Public]] [[subsidiary]],{{Unbulleted list|B3 (stock exchange)|cvm|=|23...,{{Start date and age|1999}},[[São Paulo]],Brazil,"[[Brazil]], [[Latin America]], [[Canada]]",Jean Jereissati ([[chairman]] & [[CEO]]),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"América Móvil, S.A.B. de C.V.",Logo de América Móvil.svg,100px,[[Public company|Public]],{{BMV|AMX|6024}} <br /> {{New York Stock Excha...,{{start date and age|2000|09|25|df|=|yes}},NaN,NaN,NaN,[[Carlos Slim Helú]] ([[Chairman|chairman emer...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Banco Bradesco S.A.,NaN,250px,[[S.A. (corporation)|Sociedade Anônima]],{{B3 (stock exchange)|cvm|=|906|BBDC3|BBDC4}} ...,{{start date and age|1943|03|10}} in [[Marília...,[[Osasco]],[[Brazil]],Worldwide,Luiz Carlos Trabuco Cappi <small>([[Chairman]]...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Santander Chile Holding S.A.,Banco Santander Logotipo.svg,200px,[[S.A. (corporation)|Sociedad Anónima]],{{bcs|BSANTANDER}} <br /> {{New York Stock Exc...,1978,NaN,NaN,NaN,"[[Mauricio Larraín]], <small>([[CEO]])</small>",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Banco de Chile,Banco de Chile logo.svg,NaN,[[S.A. (corporation)|Sociedad Anónima]],{{BCS|CHILE}} <br/> {{NYSE|BCH}} <br/> {{BMAD|...,October 1893,NaN,NaN,NaN,{{unbulleted list|[[Pablo Granifo Lavín]] <sma...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Banco do Brasil S.A.,Banco do Brasil Logo.svg,250,[[S.A. (corporation)|Sociedade Anônima]],{{B3 (stock exchange)|cvm|=|1023|BBAS3}} <br>[...,"[[Rio de Janeiro]], [[Captaincy of Rio de Jane...",NaN,NaN,NaN,[[Tarciana Medeiros]] ([[Chairperson|Chairwoma...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Bancolombia S.A.,Bancolombia S.A. logo.svg,250px,[[S.A. (corporation)|Sociedad Anónima]],{{BVC|BCOLOMBIA}} <br> {{nyse|CIB}},{{Start date and age|df|=|yes|1875|01|29}} (as...,NaN,NaN,"[[Colombia]], [[Cayman Islands]], [[El Salvado...","Juan Carlos Mora Uribe, ([[President (corporat...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,BRF S.A.,BRF S.A. logo.svg,150px,[[Public company|Public]],{{B3 (stock exchange)|cvm|=|16292|BRFS3}} <br>...,"{{start date and age|August 18, 1934}}",NaN,NaN,Worldwide,Lorival Nogueira Luz Jr. (CEO) <Br> [[Pedro Pa...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Motiva,Motiva.svg,NaN,[[S.A. (corporation)|Sociedade Anônima]],{{B3 (stock exchange)|cvm|=|18821|MOTV3}} <br>...,1999,NaN,NaN,NaN,Miguel Setas ([[Chief executive officer|CEO]]),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,CEMEX S.A.B. de C.V.,Cemex_logo_2023.png,250px,[[S.A. (corporation)|Sociedad Anónima Bursátil...,{{BMV|CEMEX|5203}} <br /> {{NYSE|CX}} <<br/> {...,{{start date and age|1906|df|=|yes}},NaN,NaN,Worldwide,Rogelio Zambrano Lozano<br> {{small|(Executive...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
fetch_index_infoboxes("dax")

2025-11-29 18:16:22.351 | INFO     | __main__:fetch_index_infoboxes:70 - Index %s: found %d companies
Fetching dax infoboxes: 100%|██████████| 41/41 [01:34<00:00,  2.32s/it]
2025-11-29 18:17:57.325 | INFO     | __main__:fetch_index_infoboxes:98 - Index %s: successfully fetched %d infoboxes, failed %d
2025-11-29 18:17:57.325 | WARNING  | __main__:fetch_index_infoboxes:107 - Some companies failed (%d total): %s%s
2025-11-29 18:17:57.326 | INFO     | __main__:fetch_index_infoboxes:108 - Saved infobox database to %s


,name,former_name,logo,logo_size,logo_caption,image,image_size,image_caption,type,traded_as,...,website,parent,logo_class,module,former_names,image_alt,logo_upright,trade_name,native_name,areas_served
0,Adidas AG,Gebrüder Dassler Schuhfabrik (1924–1949),Adidas 2022 logo.svg,200,Main logo since 2022,Herzogenaurach - Adidas - 2016.jpg,250,"Current factory outlet in Herzogenaurach, Germ...",[[Public company|Public]],{{FWB|ADS|isin|=|DE000A1EWWW0}} <br />[[DAX|DA...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Airbus SE,{{Indented plainlist|\n* '''Parent company:'''...,Airbus Logo 2017.svg {{!}} class=skin-invert,180px,NaN,Airbus Lagardère - Aéroconstellation.jpg,250px,"Lagardère production plant in [[Blagnac]], France",[[Public company|Public]],{{Plainlist|\n* |BMAD|isin|=|NL0000235190|AIR|...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Allianz,NaN,Allianz.svg,NaN,NaN,Wzwz_schwabing_26_allianz_building.JPG,NaN,Headquarters in Munich,[[Public company|Public]] (''[[societas Europa...,{{ubl|FWB|ALV|[[DAX]] component}} {{FWB|ALV}},...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BASF SE,NaN,BASF-Logo bw.svg,205px,NaN,NaN,NaN,NaN,[[Public company|Public]],{{ubl|FWB|BAS|isin|=|DE000BASF111|[[DAX]] comp...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Bayer AG,NaN,Logo Bayer.svg,160px,NaN,Leverkusen Kaiser-Wilhelm-Allee 0004.jpg,NaN,Headquarters in Leverkusen,[[Public company|Public]],{{ubl|class|=|nowrap|FWB|BAYN|[[DAX]] componen...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Beiersdorf AG,NaN,Beiersdorf Logo.svg {{!}} class=skin-invert-image,250px,Beiersdorf's logo used since January 2014,Beiersdorf Headquarters Hamburg 1.jpg,250px,"Headquarters in [[Hamburg]], Germany",[[Public company|Public]] ([[Aktiengesellschaf...,{{Unbulleted list | |FWB|BEI| | [[DAX]] compon...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Bayerische Motoren Werke Aktiengesellschaft,NaN,BMW logo (white + grey background square).svg,NaN,Official logo since 2020,"4 cilindros de BMW, Múnich, Alemania1.jpg",NaN,"[[BMW Headquarters]] in Munich, Germany",[[Public company|Public]],{{unbulleted list| |FWB|BMW| |[[DAX]] componen...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Brenntag SE,NaN,Brenntag Logo 2022.svg,250px,NaN,NaN,NaN,NaN,[[Public company|Public]] (''[[Societas Europa...,{{Unbulleted list |FWB|BNR| |FWB|BNRA| ([[Amer...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Commerzbank AG,NaN,Commerzbank (2009).svg,NaN,NaN,Frankfurt_Commerzbank_vom_Schaumainkai.jpg,NaN,"[[Commerzbank Tower]], the headquarters of Com...",[[Public company|Public]],{{FWB|CBK}} <br>[[DAX]],...,{{Official URL}},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Covestro AG,NaN,Covestro Logo.svg,150px,NaN,NaN,NaN,NaN,''[[Aktiengesellschaft]]'',{{plainlist|\n* |FWB|1COV|}} {{FWB|1COV}},...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:

def load_and_analyze_infobox_data(
    databases_dir: Path | str = Path("./data/databases")
) -> Dict[str, pd.DataFrame]:
    """
    Load all *_infoboxes.csv files from `databases_dir`, print basic stats,
    and return a dictionary of DataFrames keyed by index name.
    """
    databases_dir = Path(databases_dir)
    csv_files = sorted(databases_dir.glob("*_infoboxes.csv"))

    infobox_data: Dict[str, pd.DataFrame] = {}

    if not csv_files:
        print(f"⚠️ No *_infoboxes.csv files found in {databases_dir.resolve()}")
        return infobox_data

    print("📂 Loading infobox CSVs from:", databases_dir.resolve())

    for csv_path in csv_files:
        index_name = csv_path.stem.replace("_infoboxes", "")

        df = pd.read_csv(csv_path)
        infobox_data[index_name] = df

        print("\n" + "=" * 60)
        print(f" INDEX: {index_name.upper()}")
        print("=" * 60)
        print(f"- File: {csv_path.name}")
        print(f"- Shape: {len(df)} rows × {len(df.columns)} columns")
        print(f"- Example columns:")

        # Affiche les 10 premières colonnes max, plus un message si plus
        max_cols_preview = 10
        preview_cols = list(df.columns[:max_cols_preview])
        for col in preview_cols:
            print(f"  • {col}")

        remaining = len(df.columns) - len(preview_cols)
        if remaining > 0:
            print(f"  … and {remaining} more columns")

    print("\n✅ Data loaded into 'infobox_data' with indexes:")
    print("   ", ", ".join(sorted(infobox_data.keys())))

    return infobox_data


# Exécution
infobox_data = load_and_analyze_infobox_data()

📂 Loading infobox CSVs from: /content/data/databases

 INDEX: CAC40
- File: cac40_infoboxes.csv
- Shape: 37 rows × 74 columns
- Example columns:
  • name
  • logo
  • logo_alt
  • logo_size
  • foundation
  • founders
  • hq_location
  • hq_location_city
  • hq_location_country
  • type
  … and 64 more columns

 INDEX: DAX
- File: dax_infoboxes.csv
- Shape: 37 rows × 68 columns
- Example columns:
  • name
  • former_name
  • logo
  • logo_size
  • logo_caption
  • image
  • image_size
  • image_caption
  • type
  • traded_as
  … and 58 more columns

 INDEX: NASDAQ100
- File: nasdaq100_infoboxes.csv
- Shape: 98 rows × 70 columns
- Example columns:
  • name
  • logo
  • image
  • image_upright
  • image_caption
  • trading_name
  • former_name
  • type
  • traded_as
  • industry
  … and 60 more columns

 INDEX: SP500
- File: sp500_infoboxes.csv
- Shape: 474 rows × 120 columns
- Example columns:
  • name
  • logo
  • logo_size
  • image
  • image_size
  • image_caption
  • former_name
  •

In [26]:
from typing import Any, Dict
import re
import pandas as pd


# ============================================================================
# TEXT CLEANING & CONTEXT BUILDING FOR LLM
# ============================================================================

def clean_text(text: Any) -> str:
    """
    Clean and normalize text for LLM input.
    Removes Wikipedia markup and normalizes whitespace.
    """
    if pd.isna(text) or text is None:
        return ""

    text = str(text)

    # Handle common Wikipedia templates by extracting useful content
    # {{US$|24.58 billion|...}} -> $24.58 billion
    text = re.sub(r"\{\{US\$\|([^}|]+)[^}]*\}\}", r"$\1", text)

    # {{circa|61,500}} -> ~61,500
    text = re.sub(r"\{\{circa\|([^}|]+)[^}]*\}\}", r"~\1", text)

    # {{increase}}, {{decrease}} -> arrows
    text = re.sub(r"\{\{increase\}\}", "↑", text)
    text = re.sub(r"\{\{decrease\}\}", "↓", text)

    # {{Start date and age|1902|6|13}} -> 1902-06-13
    text = re.sub(
        r"\{\{Start date and age\|(\d+)\|(\d+)\|(\d+)[^}]*\}\}",
        r"\1-\2-\3",
        text,
    )

    # {{URL|example.com}} -> example.com
    text = re.sub(r"\{\{URL\|([^}|]+)[^}]*\}\}", r"\1", text)

    # {{plainlist|...}} and {{Unbulleted list|...}} - extract items
    text = re.sub(r"\{\{(?:plainlist|Unbulleted list)\|", "", text)

    # Remove remaining templates iteratively (handles nesting)
    prev_text = ""
    while prev_text != text:
        prev_text = text
        text = re.sub(r"\{\{[^{}]*\}\}", "", text)

    # Extract text from wiki links [[text|display]] -> display or [[text]] -> text
    text = re.sub(r"\[\[([^|\]]*\|)?([^\]]+)\]\]", r"\2", text)

    # Remove reference tags
    text = re.sub(r"<ref[^>]*>.*?</ref>", "", text, flags=re.DOTALL)
    text = re.sub(r"<ref[^/]*/?>", "", text)
    text = re.sub(r"</ref>", "", text)
    text = re.sub(r"<[^>]+>", "", text)

    # Clean up remaining brackets, braces, pipes, asterisks
    text = re.sub(r"[\[\]{}|*]", "", text)

    # Normalize whitespace
    text = re.sub(r"\s+", " ", text).strip()

    return text


def extract_key_facts(row: pd.Series) -> Dict[str, str]:
    """
    Extract key facts from a company infobox row.
    Uses case-insensitive matching to handle Wikipedia's varying field names.
    """
    field_mappings = {
        "name": ["name", "_company_name", "company_name"],
        "type": ["type"],
        "industry": ["industry", "industries"],
        "founded": ["foundation", "founded", "established"],
        "founder": ["founder", "founders"],
        "headquarters": [
            "hq_location",
            "headquarters",
            "location_city",
            "hq_location_city",
        ],
        "country": ["location_country", "hq_location_country", "country"],
        "key_people": ["key_people", "ceo", "chairman"],
        "employees": ["num_employees", "employees"],
        "revenue": ["revenue"],
        "website": ["website", "url", "homepage"],
    }

    # Create lowercase column mapping
    col_lower = {col.lower(): col for col in row.index}

    facts: Dict[str, str] = {}
    for field, possible_names in field_mappings.items():
        for name in possible_names:
            if name in col_lower:
                value = row[col_lower[name]]
                cleaned = clean_text(value)
                if cleaned:
                    facts[field] = cleaned
                    break

    return facts


def row_to_context(row: pd.Series) -> str:
    """
    Convert a company row into a formatted context string for LLM input.
    """
    facts = extract_key_facts(row)

    if not facts:
        return "No company information available."

    lines = ["COMPANY INFORMATION", "-" * 40]

    field_order = [
        "name",
        "type",
        "industry",
        "founded",
        "founder",
        "headquarters",
        "country",
        "key_people",
        "employees",
        "revenue",
        "website",
    ]

    for field in field_order:
        if field in facts:
            label = field.replace("_", " ").title()
            lines.append(f"{label}: {facts[field]}")

    return "\n".join(lines)


# ============================================================================
# HIGHER-LEVEL WRAPPERS
# ============================================================================

def add_llm_context_column(df: pd.DataFrame) -> pd.DataFrame:
    """
    Return a copy of df with an extra column 'llm_context'
    containing the formatted context for each row.
    """
    df = df.copy()
    df["llm_context"] = df.apply(row_to_context, axis=1)
    return df


def preprocess_infobox_dict(
    infobox_data: Dict[str, pd.DataFrame]
) -> Dict[str, pd.DataFrame]:
    """
    Given a dict of {index_name: DataFrame}, return a new dict where
    each DataFrame has an added 'llm_context' column.
    """
    return {name: add_llm_context_column(df) for name, df in infobox_data.items()}


# ============================================================================
# DEMO / PREVIEW UTIL
# ============================================================================

def demo_preprocessing_for_index(
    infobox_data: Dict[str, pd.DataFrame],
    index_name: str = "sp500",
) -> None:
    """
    Show how cleaning and context-building work on the first row
    of a given index:
      - raw vs cleaned fields
      - extracted key facts
      - final LLM context block
    """
    if index_name not in infobox_data:
        print(
            f"❌ Index '{index_name}' not found in infobox_data.\n"
            f"   Available indexes: {list(infobox_data.keys())}"
        )
        return

    df = infobox_data[index_name]
    if df.empty:
        print(f"⚠️ DataFrame for index '{index_name}' is empty.")
        return

    sample_row = df.iloc[0]

    print("\n" + "=" * 70)
    print(f" DEMO: PREPROCESSING FOR INDEX '{index_name.upper()}'")
    print("=" * 70)

    # 1) Raw vs cleaned example fields
    print("\n[1] RAW vs CLEANED FIELDS")
    print("-" * 70)
    for col in ["revenue", "num_employees", "foundation"]:
        if col in sample_row.index:
            print(f"\n• Column: {col}")
            print(f"  Raw:    {sample_row[col]}")
            print(f"  Cleaned:{' ' if clean_text(sample_row[col]) else ''}{clean_text(sample_row[col])}")

    # 2) Extracted key facts
    print("\n" + "-" * 70)
    print("[2] EXTRACTED KEY FACTS")
    print("-" * 70)
    facts = extract_key_facts(sample_row)
    if not facts:
        print("No key facts extracted.")
    else:
        for k, v in facts.items():
            print(f"  - {k}: {v}")

    # 3) Final LLM context
    print("\n" + "-" * 70)
    print("[3] FORMATTED CONTEXT FOR LLM")
    print("-" * 70)
    context = row_to_context(sample_row)
    print(context)

    print("\n" + "=" * 70)
    print(" END OF DEMO")
    print("=" * 70)


# ============================================================================
# TEST / USAGE EXAMPLE
# ============================================================================

# Supposons que tu aies déjà :
# infobox_data = load_and_analyze_infobox_data()

# 1. Add LLM context column to a single dataframe
sp500_with_context = add_llm_context_column(infobox_data["sp500"])

# 2. Add LLM context to all indexes in one go
infobox_with_context = preprocess_infobox_dict(infobox_data)

# 3. Run a demo on SP500
demo_preprocessing_for_index(infobox_data, index_name="sp500")



 DEMO: PREPROCESSING FOR INDEX 'SP500'

[1] RAW vs CLEANED FIELDS
----------------------------------------------------------------------

• Column: revenue
  Raw:    {{decrease}} {{US$|24.58 billion|link|=|yes}} (2024)
  Cleaned: ↓ $24.58 billion (2024)

• Column: num_employees
  Raw:    {{circa|61,500}} (2024)
  Cleaned: ~61,500 (2024)

• Column: foundation
  Raw:    {{Start date and age|1902|6|13}} in [[Two Harbors, Minnesota]], U.S.
  Cleaned: 1902-6-13 in Two Harbors, Minnesota, U.S.

----------------------------------------------------------------------
[2] EXTRACTED KEY FACTS
----------------------------------------------------------------------
  - name: 3M Company
  - type: Public
  - industry: Conglomerate
  - founded: 1902-6-13 in Two Harbors, Minnesota, U.S.
  - founder: J. Danley BuddHenry S. BryanWilliam A. McGonagleJohn DwanHermon W. Cable Charles Simmonsref
  - headquarters: Maplewood, Minnesota
  - country: U.S.
  - key_people: Michael F. Roman (chairman) William M. Br

In [31]:
from typing import List


# ============================================================================
# PROMPT TEMPLATES FOR LLM TASKS
# ============================================================================

def qa_prompt(context: str, question: str) -> str:
    """
    Crée un prompt de Question/Réponse sur une entreprise.
    """
    return f"""You are a helpful assistant answering questions about companies.

{context}

Question: {question}

Please provide a clear, concise answer based only on the information provided above. If the information is not available, say so."""


def classification_prompt(context: str, categories: List[str]) -> str:
    """
    Crée un prompt pour classer une entreprise dans une catégorie.
    """
    categories_str = "\n".join(f"- {cat}" for cat in categories)

    return f"""You are a business analyst classifying companies into categories.

{context}

Based on the company information above, classify this company into ONE of the following categories:
{categories_str}

Respond with ONLY the category name, followed by a brief one-sentence justification."""


def summarization_prompt(context: str) -> str:
    """
    Crée un prompt de résumé d'informations d’entreprise.
    """
    return f"""You are a business analyst creating company summaries.

{context}

Please provide a concise 2-3 sentence summary of this company, highlighting:
1. What the company does (industry/business)
2. Key characteristics (size, location, notable facts)
3. Any distinguishing features

Keep the summary factual and professional."""


def comparison_prompt(context1: str, context2: str) -> str:
    """
    Crée un prompt de comparaison entre deux entreprises.
    """
    return f"""You are a business analyst comparing companies.

=== COMPANY 1 ===
{context1}

=== COMPANY 2 ===
{context2}

Please compare these two companies across the following dimensions:
1. Industry & Business Focus
2. Size & Scale (employees, revenue if available)
3. Geographic Presence
4. Key Similarities
5. Key Differences

Provide a structured comparison with clear insights."""


def extraction_prompt(context: str, fields: List[str]) -> str:
    """
    Crée un prompt pour extraire certains champs sous forme de JSON.
    """
    fields_str = ", ".join(f'"{f}"' for f in fields)

    return f"""You are a data extraction specialist.

{context}

Extract the following fields from the company information above:
[{fields_str}]

Respond with a valid JSON object containing only these fields.
Use null for any field that cannot be determined from the information provided.

Example format:
{{
  "field1": "value1",
  "field2": "value2"
}}"""


# ============================================================================
# DEMO : show examples of prompts for one index
# ============================================================================

def _show_block(title: str):
    print("\n" + "=" * 60)
    print(f"{title}")
    print("=" * 60 + "\n")


def demo_prompts_for_index(
    infobox_data,
    index_name: str = "sp500",
):
    """
    Affiche des exemples de prompts (Q&A, classification, résumé, extraction,
    comparaison) construits à partir des premières lignes d'un index.
    """
    if index_name not in infobox_data:
        print(f"❌ Index '{index_name}' not found. Available: {list(infobox_data.keys())}")
        return

    df = infobox_data[index_name]
    if df.empty:
        print(f"⚠️ DataFrame for index '{index_name}' is empty.")
        return

    # Contexte pour la première entreprise
    sample_context = row_to_context(df.iloc[0])

    print("\n" + "=" * 60)
    print(f" DEMO PROMPTS FOR INDEX: {index_name.upper()}")
    print("=" * 60)

    # 1. Q&A
    _show_block("[1] Q&A PROMPT")
    qa = qa_prompt(sample_context, "What industry is this company in?")
    print(qa)

    # 2. Classification
    _show_block("[2] CLASSIFICATION PROMPT")
    categories = ["Technology", "Healthcare", "Finance", "Industrial", "Consumer Goods"]
    classification = classification_prompt(sample_context, categories)
    print(classification)

    # 3. Summarization
    _show_block("[3] SUMMARIZATION PROMPT")
    summary = summarization_prompt(sample_context)
    print(summary)

    # 4. Extraction
    _show_block("[4] EXTRACTION PROMPT")
    fields = ["company_name", "industry", "employee_count", "annual_revenue"]
    extraction = extraction_prompt(sample_context, fields)
    print(extraction)

    # 5. Comparison (needs at least 2 companies)
    if len(df) >= 2:
        _show_block("[5] COMPARISON PROMPT")
        context2 = row_to_context(df.iloc[1])
        comparison = comparison_prompt(sample_context, context2)
        print(comparison)
    else:
        print("\n⚠️ Not enough rows for comparison prompt (need at least 2).")

    print("\n" + "=" * 60)
    print(" END OF PROMPT DEMO")
    print("=" * 60)


# ============================================================================
# USAGE EXAMPLE (in Colab)
# ============================================================================

# Suppose you already have:
# - infobox_data = load_and_analyze_infobox_data()
# - row_to_context defined

demo_prompts_for_index(infobox_data, index_name="sp500")



 DEMO PROMPTS FOR INDEX: SP500

[1] Q&A PROMPT

You are a helpful assistant answering questions about companies.

COMPANY INFORMATION
----------------------------------------
Name: 3M Company
Type: Public
Industry: Conglomerate
Founded: 1902-6-13 in Two Harbors, Minnesota, U.S.
Founder: J. Danley BuddHenry S. BryanWilliam A. McGonagleJohn DwanHermon W. Cable Charles Simmonsref
Headquarters: Maplewood, Minnesota
Country: U.S.
Key People: Michael F. Roman (chairman) William M. Brown (CEO)ref
Employees: ~61,500 (2024)
Revenue: ↓ $24.58 billion (2024)
Website: 3m.com

Question: What industry is this company in?

Please provide a clear, concise answer based only on the information provided above. If the information is not available, say so.

[2] CLASSIFICATION PROMPT

You are a business analyst classifying companies into categories.

COMPANY INFORMATION
----------------------------------------
Name: 3M Company
Type: Public
Industry: Conglomerate
Founded: 1902-6-13 in Two Harbors, Minnesota

Create a PromptBuilder class with different methods for creating prompts:
- qa prompt
- classification prompt
- summarization prompt
- comparison prompt
- information extraction prompt

In [32]:
"""
LLM BEST PRACTICES — CLEAN OVERVIEW
-----------------------------------

1. Context Preparation
   - Keep context focused, relevant, and clean
   - Standardize text and remove ambiguous parts
   - Add source metadata when available

2. Prompt Design
   - Use clear, specific task instructions
   - Include examples where helpful
   - Define expected output formats precisely
   - Assign a model role for better consistency

Applications of Company Data:
   - Industry classification
   - Market and competitor analysis
   - Risk assessment
   - Investment profiling
   - Data enrichment from Wikipedia
   - Report and summary generation
   - Extraction of structured information
   - Reputation and sentiment insights
   - Trend and pattern detection
   - Peer comparison and benchmarking
"""


'\nLLM BEST PRACTICES — CLEAN OVERVIEW\n-----------------------------------\n\n1. Context Preparation\n   - Keep context focused, relevant, and clean\n   - Standardize text and remove ambiguous parts\n   - Add source metadata when available\n\n2. Prompt Design\n   - Use clear, specific task instructions\n   - Include examples where helpful\n   - Define expected output formats precisely\n   - Assign a model role for better consistency\n\nApplications of Company Data:\n   - Industry classification\n   - Market and competitor analysis\n   - Risk assessment\n   - Investment profiling\n   - Data enrichment from Wikipedia\n   - Report and summary generation\n   - Extraction of structured information\n   - Reputation and sentiment insights\n   - Trend and pattern detection\n   - Peer comparison and benchmarking\n'

🚀 NEXT STEPS (in anticipation of the final lab)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1. Export your full dataset using PromptExporter
2. Test prompts with a small sample (5-10 companies)
3. Evaluate LLM outputs for quality and accuracy
4. Iterate on prompts based on results
5. Scale up to full dataset using batch APIs
6. Monitor token usage and costs
7. Implement feedback loops for continuous improvement
8. Build evaluation metrics for output quality